In [1]:
import pandas as pd
from nltk.corpus import stopwords

In [2]:
df = pd.read_excel('../data/processed/LNB_processed.xlsx')

In [3]:
text = df.sample(1)

In [14]:
txt = text.full_text.values[0]

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krister\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
stops = set(stopwords.words('german'))

In [16]:
import spacy
nlp = spacy.load('../data/models/spacy_model_250421/')

In [24]:
stops = set(stopwords.words('german'))

def clean_text(text):
    
    finaltext = ''
    doc = nlp(text)
    
    for token in doc:
        if token.text not in stops:
            finaltext += token.lemma_
            finaltext += ' '
            
    return finaltext

test = clean_text(txt)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string

In [27]:
 vectorizer = TfidfVectorizer(lowercase=True,
                              max_features=300,
                              max_df=0.8,
                              min_df=2,
                              ngram_range=(1,3))

In [36]:
cleaned_docs =  [clean_text(txt) for txt in df.sample(100).full_text]

In [37]:
cleaned_docs

['Vermischtes . \n\n — AuS Athen telegraphisch ge \n melden : der Kronprinz Kronprinzessin \n begeben fich Feier Hochzeit \n Prinzessin Margarethe Preuße Berlin , \n Kronprinzessin freudig Ereignis \n entgegensehen . \n — der „ Ueberfall Straßburg " . Ein \n Ueberfall , alt Ziethen Ehre machen \n haben , , weit entfernen , feindselig , \n Sturm drr Freude entfesseln , \n Straßburg Besuch deS deutsch Kaiser » . der \n Kaiser Pferd Zug mitge \n brechen , niemand Mitwisien \n schaft anweisen , geheim \n Absicht , Straßburg heimlich Besuch \n abstatten , halten , Umgebung \n eher davon erfahren , kaiserliche \n Zug Bahnhof Appenweier Befehl \n Kaiser anf daS Straßburger GeleiS übergeführt \n werden . In Kehl Hofzug \n Mittwoch Straßburg 12 Uhr 32 Minute \n ankommend Personenzug einholen . der Bahn- \n Hofsvorsteher Kehl werden Hofzug ge- \n rufen erhalten Befehl , Ankunft \n Kaiser Straßburg melden . der \n Hofzug treffen 12 Uhr 40 Minute . \n der Kaiser bleiben Wagen , Pferd aus- \n laden

In [42]:
for doc in cleaned_docs:
    doc = doc.replace('\n', '')

In [50]:
vectors = vectorizer.fit_transform(cleaned_docs)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = vectors.todense()

In [76]:
all_keywords = []

for text in denselist:
    ct = 0
    keywords = []
    for word in text.flat:
        if word > 0:
            keywords.append(feature_names[ct])
        ct += 1
    all_keywords.append(keywords)

In [86]:
true_k = 20
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(vectors)

KMeans(max_iter=100, n_clusters=20, n_init=1)

In [93]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

with open('../temp/trc_test.txt', 'w', encoding='utf8') as f:
    for i in range(true_k):
        f.write(f'Cluster {i}')
        f.write('\n')
        for ind in order_centroids[i, :10]:
            f.write(' %s' %terms[ind])
            f.write('\n')
        f.write('\n')
        f.write('\n')